In [ ]:
!pip install --quiet gradio opencv-python moviepy numpy mediapipe Pillow

In [ ]:
import os
import cv2
import numpy as np
import gradio as gr
import mediapipe as mp
from PIL import Image, ImageOps
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import random

# Initialisation des détecteurs de visage et de points de repère faciaux
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def align_and_fit_face(image_rgb, output_size=(1280, 720), eye_target=(640, 360)):
    """
    Aligne les yeux sur un point cible tout en s'assurant que l'image entière est visible
    dans le cadre de sortie, sans recadrage (zoom < 1).
    Note: eye_target est légèrement abaissé pour un meilleur centrage vertical.
    """
    h, w, _ = image_rgb.shape
    out_w, out_h = output_size
    img_for_detection = image_rgb.astype(np.uint8)

    # Détection des yeux avec MediaPipe
    res = mp_face_mesh.process(cv2.cvtColor(img_for_detection, cv2.COLOR_BGR2RGB))
    left_eye, right_eye = None, None
    if res and res.multi_face_landmarks:
        lm = res.multi_face_landmarks[0].landmark
        left_idxs = [33, 133]
        right_idxs = [362, 263]
        left_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in left_idxs], axis=0)
        right_eye = np.mean([(lm[i].x * w, lm[i].y * h) for i in right_idxs], axis=0)

    # Fallback sur Haar Cascade si MediaPipe échoue
    if left_eye is None or right_eye is None:
        gray = cv2.cvtColor(img_for_detection, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        if len(faces) > 0:
            x, y, wb, hb = max(faces, key=lambda f: f[2]*f[3])
            cx, cy = x + wb/2, y + hb/3.0
            left_eye = np.array([cx - wb*0.18, cy])
            right_eye = np.array([cx + wb*0.18, cy])

    # Si aucun visage n'est détecté, on centre simplement l'image
    if left_eye is None or right_eye is None:
        scale = min(out_w / w, out_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = cv2.resize(image_rgb, (new_w, new_h), interpolation=cv2.INTER_AREA)
        final_image = np.full((out_h, out_w, 3), 255, np.uint8)
        x_offset = (out_w - new_w) // 2
        y_offset = (out_h - new_h) // 2
        final_image[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
        return final_image

    # Calcul de la transformation
    eyes_center = (np.array(left_eye) + np.array(right_eye)) / 2.0
    dx, dy = right_eye[0] - left_eye[0], right_eye[1] - left_eye[1]
    angle = np.degrees(np.arctan2(dy, dx))
    scale = min(out_w / w, out_h / h)
    M = cv2.getRotationMatrix2D(tuple(eyes_center), angle, scale)
    rotated_eyes_center = np.dot(M, np.array([eyes_center[0], eyes_center[1], 1]))
    M[0, 2] += eye_target[0] - rotated_eyes_center[0]
    M[1, 2] += eye_target[1] - rotated_eyes_center[1]
    
    # Appliquer la transformation finale
    warped = cv2.warpAffine(image_rgb, M, (out_w, out_h),
                            flags=cv2.INTER_LINEAR, 
                            borderMode=cv2.BORDER_CONSTANT, 
                            borderValue=(255, 255, 255))
    return warped

def create_face_movie(files, music=None, duration_per_face=5.0, fade_duration=2.5):
    os.makedirs('outputs', exist_ok=True)
    clips = []
    for f in files:
        try:
            pil_img = Image.open(f.name).convert('RGBA')
        except Exception as e:
            print(f"Impossible d'ouvrir le fichier {f.name}: {e}")
            continue

        # --- ORDRE DES OPÉRATIONS CORRIGÉ ---
        # 1. Appliquer une inclinaison aléatoire à l'image originale
        rot_angle = random.uniform(-5, 5)
        tilted_pil = pil_img.rotate(rot_angle, expand=True, resample=Image.BICUBIC)

        # 2. Créer un fond blanc et coller l'image inclinée dessus pour gérer la transparence
        white_bg = Image.new("RGBA", tilted_pil.size, (255, 255, 255, 255))
        white_bg.paste(tilted_pil, (0, 0), tilted_pil)
        white_bg = white_bg.convert('RGB')

        # 3. Ajouter une bordure blanche épaisse pour l'effet 'photo papier'
        bordered_pil = ImageOps.expand(white_bg, border=60, fill='white')

        # 4. Convertir en format NumPy pour la détection et l'alignement
        img_to_align_np = np.array(bordered_pil)

        # 5. Aligner cette nouvelle "photo-objet" complète
        final_frame_np = align_and_fit_face(img_to_align_np)
        if final_frame_np is None:
            continue
        
        clip = ImageClip(final_frame_np).set_duration(duration_per_face)
        clips.append(clip)

    if not clips:
        return "Aucune image utilisable n'a été traitée.", None, None

    final_clip = concatenate_videoclips(clips, padding=-fade_duration, method="compose")

    if music is not None:
        try:
            audio = AudioFileClip(music.name).set_duration(final_clip.duration)
            final_clip = final_clip.set_audio(audio)
        except Exception as e:
            print(f"Erreur audio: {e}")
            pass

    out_path = 'outputs/movie_aligned_stack_CORRECTED.mp4'
    final_clip.write_videofile(out_path, fps=24, codec='libx264', audio_codec='aac', ffmpeg_params=['-pix_fmt', 'yuv420p'])
    return f"Vidéo générée avec {len(clips)} images.", out_path, out_path

iface = gr.Interface(
    fn=create_face_movie,
    inputs=[
        gr.File(label="Images (plusieurs fichiers acceptés)", file_types=["image"], file_count="multiple"),
        gr.File(label="Musique (optionnel)", file_types=["audio"])
    ],
    outputs=[
        gr.Textbox(label="Statut", lines=5),
        gr.Video(label="Vidéo Résultat"),
        gr.File(label="Télécharger la Vidéo")
    ],
    title="Créateur de Diaporama Vidéo (Alignement des Yeux)",
    description="CORRIGÉ : Applique une bordure et une inclinaison à chaque photo, PUIS aligne le tout pour que les yeux restent fixes. L'image entière est conservée sans recadrage.",
    allow_flagging='never'
)
iface.launch(share=True, debug=True)
